In [1]:
import time

from astroscrappy import detect_cosmics
from lacosmic import lacosmic
from PyCosmic import det_cosmics

import logging

from scrs import Image
from scrs.constants import OUT_DIR

In [2]:
logging.basicConfig()
logging.getLogger().setLevel(logging.WARNING)

logger = logging.getLogger()

In [3]:
files = list((OUT_DIR / "auto" / "raw").glob("*_raw.fits"))
num_files = len(files)

In [ ]:
for i, file in enumerate(files):
    # Run all three algorithms. Note that parameters applied are not optimised,
    # but are instead set to align based on the defaults of the three algorithms.

    msg = f"Processing {i+1}/{num_files}: {file.stem}."
    print(msg, end="\r")

    img = Image().load_fits(file)
    mask_file = OUT_DIR / "auto" / "masks" / file.name.replace("raw", "mask")

    masks_img = Image().enable_history(mask_file)

    data = img.data.astype(float)  # Required for NaN masking used by lacosmic

    print(f"{msg} Algorithm: LA Cosmic", end="\r")
    t0 = time.time()
    lc_clean, lc_mask = lacosmic(
        data,
        contrast=5,
        cr_threshold=4.5,
        neighbor_threshold=0.3,
        effective_gain=1,
        readnoise=2,
    )
    dur = time.time() - t0
    masks_img.set_data(lc_mask.astype(int)).add_to_header(LC_TIME=dur)
    del lc_clean, lc_mask

    print(f"{msg} Algorithm: Astroscrappy", end="\r")
    t0 = time.time()
    ac_mask, as_clean = detect_cosmics(data, readnoise=2)
    dur = time.time() - t0
    masks_img.set_data(ac_mask.astype(int)).add_to_header(AC_TIME=dur)
    del as_clean

    print(f"{msg} Algorithm: PyCosmic", end="\r" if i < num_files - 1 else "\n")
    to = time.time()
    pc_img = det_cosmics(data, iterations=4, rdnoise=2)
    dur = time.time() - to
    masks_img.set_data(pc_img.mask.astype(int)).add_to_header(PC_TIME=dur)
    del pc_img